In [ ]:
# SETUP the colab Env.

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [ ]:
# Clone github repository setup
# import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'MyDrive/Colab Notebooks/'  
# Replace with your github repository
GIT_REPOSITORY = "promo-recommendation" 
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path 
# !mkdir "{PROJECT_PATH}"    

GIT_PATH = "https://" + "@github.com/" + "fellowship/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive/MyDrive/Colab Notebooks/
GIT_PATH:  https://@github.com/fellowship/promo-recommendation.git


In [ ]:
%cd "{PROJECT_PATH}{"promo-recommendation/"}"

/content/drive/MyDrive/Colab Notebooks/promo-recommendation


In [ ]:
!git branch -av

* deep_and_wide                          277ba41 directory change
  main                                   401931c Merge pull request #4 from fellowship/data_generation_refine
  remotes/origin/HEAD                    -> origin/main
  remotes/origin/cohort_var_xgb          025362d initial implementation of cohort variance analysis
  remotes/origin/deep_and_wide           277ba41 directory change
  remotes/origin/initial_data_generation 5a736b5 add script and plots for example data
  remotes/origin/main                    401931c Merge pull request #4 from fellowship/data_generation_refine
  remotes/origin/setup                   a15a8c2 update readme
  remotes/origin/uneven_cohort_fix       85cb0fe fix a sampling frequency bug where number of elements to modify is greater than total number of elements


In [ ]:
!git branch

* deep_and_wide
  main


In [ ]:
!python ./00.generate_data.py

In [ ]:
import gc
import importlib
import os
from routine.utilities import generate_CSV, df_to_dataloader, generate_feature_columns
from routine.data_generation import generate_data
from routine.models import build_wide_model, build_deep_model, build_wide_and_deep_model, \
    build_bayesian_model, evaluate_bandit
from os.path import exists
from pprint import pprint
import tensorflow as tf
import sys
import numpy as np
import pandas as pd

In [ ]:
data_regenerate = False
if data_regenerate:
    obs_df, user_df, camp_df = generate_data(
        num_users=100,
        num_campaigns=10,
        samples_per_campaign=100,
        num_cohort=10,
        cohort_variances=np.linspace(0.05, 0.6, 10),
        fh_cohort=True,
        response_sig_a=10,
        even_cohort=True,
        cross_response=True
    )
else:
    obs_df = pd.read_csv('observation_odd.csv')


INPUT_DATA_PATH = './deep_and_wide/NN_Inputs/input_data'
if not os.path.isdir(INPUT_DATA_PATH):
    os.makedirs(INPUT_DATA_PATH)

[INFO] The training dataframe has 601 instances from indices 0 to 599
[INFO] The validation dataframe has 201 instances from indices 600 to 799
[INFO] The training dataframe has 200 instances from indices 800 to 1000


In [ ]:
train_path = INPUT_DATA_PATH + "/train.csv"
val_path = INPUT_DATA_PATH + "/val.csv"
test_path = INPUT_DATA_PATH + "/test.csv"
re_create = True
if re_create:
    generate_CSV(obs_df,
                 train_path,
                 val_path,
                 test_path,
                 verbose=True)

[INFO] Train dataloader:
<BatchDataset element_spec=({'user_id': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'camp_id': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'cohort': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'user_f0': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'user_f1': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'user_fh': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'camp_f0': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'camp_f1': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'camp_fh': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}, TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>
[INFO] Val dataloader:
<BatchDataset element_spec=({'user_id': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'camp_id': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'cohort': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'user_f0': TensorSpec(shape=(None,), dtype=

In [ ]:
batch_size = 100
n_epochs = 100
feature_columns = ["user_id", "camp_id", "cohort",
                   "user_f0", "user_f1", "user_fh",
                   "camp_f0", "camp_f1", "camp_fh"]
target_column = "response"

train_dl = df_to_dataloader(train_path,
                            feature_columns,
                            target_column,
                            batch_size=batch_size)
val_dl = df_to_dataloader(val_path,
                          feature_columns,
                          target_column,
                          batch_size=batch_size)
test_dl = df_to_dataloader(test_path,
                           feature_columns,
                           target_column,
                           shuffle=False,
                           batch_size=batch_size)

print("[INFO] Train dataloader:")
pprint(train_dl)
print("[INFO] Val dataloader:")
pprint(val_dl)
print("[INFO] Test dataloader:")
pprint(test_dl)


88

In [ ]:
feature_column_dict, feature_column_input_dict = generate_feature_columns()
# defining the input to be fed into each model
inputs = {**feature_column_input_dict["numeric"], **feature_column_input_dict["embedding"]}


In [ ]:
models_dir = './deep_and_wide/NN_checkpoint'
if not os.path.isdir(models_dir):
    os.makedirs(models_dir)
# create the folders to save the checkpoints
wmodel_dir = models_dir + '/Wide'
dmodel_dir = models_dir + '/Deep'
wdmodel_dir = models_dir + '/W&D'
bayesian_dir = models_dir + '/Bayesian'
os.makedirs(wmodel_dir, exist_ok=True)
os.makedirs(dmodel_dir, exist_ok=True)
os.makedirs(wdmodel_dir, exist_ok=True)
os.makedirs(bayesian_dir, exist_ok=True)
# setting the hyperparameters
lr = 1e-3
gc.collect()

In [ ]:
# WIDE MODEL ONLY

In [ ]:
wmodel, wmodel_path, w_es, w_mc = build_wide_model(feature_column_dict,
                                                   inputs,
                                                   wmodel_dir=wmodel_dir)
wmodel.summary()  # To display the architecture

In [ ]:
again_training = True
if again_training:
    # create callback for model saving
    w_m = tf.keras.callbacks.ModelCheckpoint(
        filepath=wmodel_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        mode='min')
    H = wmodel.fit(train_dl,
                   batch_size=batch_size,
                   epochs=n_epochs,
                   validation_data=val_dl,
                   shuffle=False,
                   validation_batch_size=batch_size,
                   callbacks=[w_es, w_mc, w_m])
else:
    wmodel = tf.keras.models.load_model(wmodel_path)

In [ ]:
eval_wmodel_train = wmodel.evaluate(train_dl)
eval_wmodel_val = wmodel.evaluate(val_dl)
eval_wmodel_test = wmodel.evaluate(test_dl)
# Print the results
print("\n[INFO] On Training Set:")
print(eval_wmodel_train)
print("\n[INFO] On Validation Set:")
print(eval_wmodel_val)
print("\n[INFO] On Test Set:")
print(eval_wmodel_test)

In [ ]:
# DEEP MODEL ONLY

In [ ]:
## With only embeddings

In [ ]:
dmodel_1_emb, dmodel_1_emb_path, d1_es, d1_mc = build_deep_model(feature_column_dict["embedding"],
                                                                 inputs,
                                                                 dmodel_dir,
                                                                 name="dmodel_1_emb.h5",
                                                                 ckpt_name="dmodel_1_emb_checkpoint.h5")
dmodel_1_emb.summary()  # To display the architecture

In [ ]:
again_training = True
if again_training:
    # create callback for model saving
    d1_m = tf.keras.callbacks.ModelCheckpoint(
        filepath=dmodel_1_emb_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        mode='min')
    H1 = dmodel_1_emb.fit(train_dl,
                          batch_size=batch_size,
                          epochs=n_epochs,
                          validation_data=val_dl,
                          shuffle=False,
                          validation_batch_size=batch_size,
                          callbacks=[d1_es, d1_mc, d1_m])
else:
    dmodel_1_emb = tf.keras.models.load_model(dmodel_1_emb_path)

In [ ]:
eval_dmodel_1_emb_train = dmodel_1_emb.evaluate(train_dl, batch_size=batch_size)
eval_dmodel_1_emb_val = dmodel_1_emb.evaluate(val_dl, batch_size=batch_size)
eval_dmodel_1_emb_test = dmodel_1_emb.evaluate(test_dl, batch_size=batch_size)
# Print the results
print("\n[INFO] On Training Set:")
print(eval_dmodel_1_emb_train)
print("\n[INFO] On Validation Set:")
print(eval_dmodel_1_emb_val)
print("\n[INFO] On Test Set:")
print(eval_dmodel_1_emb_test)

In [ ]:
## With only numerical features

In [ ]:
dmodel_2_num, dmodel_2_num_path, d2_es, d2_mc = build_deep_model(feature_column_dict["numeric"],
                                                                 inputs,
                                                                 dmodel_dir,
                                                                 name="dmodel_2_num.h5",
                                                                 ckpt_name="dmodel_2_num_checkpoint.h5")
dmodel_2_num.summary()


In [ ]:
again_training = True
if again_training:
    # create callback for model saving
    d2_m = tf.keras.callbacks.ModelCheckpoint(
        filepath=dmodel_2_num_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        mode='min')
    H2 = dmodel_2_num.fit(train_dl,
                          batch_size=batch_size,
                          epochs=n_epochs,
                          validation_data=val_dl,
                          shuffle=False,
                          validation_batch_size=batch_size,
                          callbacks=[d2_es, d2_mc, d2_m])
else:
    dmodel_2_num = tf.keras.models.load_model(dmodel_2_num_path)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 camp_f0 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 camp_f1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 camp_id (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 user_f0 (InputLayer)           [(None,)]            0           []                               
                                                                                              

In [ ]:
eval_dmodel_2_num_train = dmodel_2_num.evaluate(train_dl, batch_size=batch_size)
eval_dmodel_2_num_val = dmodel_2_num.evaluate(val_dl, batch_size=batch_size)
eval_dmodel_2_num_test = dmodel_2_num.evaluate(test_dl, batch_size=batch_size)
# Print the results
print("\n[INFO] On Training Set:")
print(eval_dmodel_2_num_train)
print("\n[INFO] On Validation Set:")
print(eval_dmodel_2_num_val)
print("\n[INFO] On Test Set:")
print(eval_dmodel_2_num_test)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['cohort', 'user_fh', 'camp_fh'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


5/7 [====================>.........] - ETA: 0s - loss: 2.2138 - accuracy: 0.3440 - auc: 0.7955 
Epoch 1: val_accuracy improved from -inf to 0.58209, saving model to ./deep_and_wide/NN_checkpoint/W&D/wdmodel_checkpoint.h5

Epoch 1: val_loss improved from inf to 1.82258, saving model to ./deep_and_wide/NN_checkpoint/W&D/wdmodel.h5
7/7 [==============================] - 4s 276ms/step - loss: 2.1809 - accuracy: 0.3727 - auc: 0.8281 - val_loss: 1.8226 - val_accuracy: 0.5821 - val_auc: 0.9567
Epoch 2/100
5/7 [====================>.........] - ETA: 0s - loss: 1.5726 - accuracy: 0.5800 - auc: 0.9523
Epoch 2: val_accuracy did not improve from 0.58209

Epoch 2: val_loss improved from 1.82258 to 0.93429, saving model to ./deep_and_wide/NN_checkpoint/W&D/wdmodel.h5
7/7 [==============================] - 0s 35ms/step - loss: 1.5016 - accuracy: 0.5740 - auc: 0.9512 - val_loss: 0.9343 - val_accuracy: 0.5672 - val_auc: 0.9590
Epoch 3/100
6/7 [========================>.....] - ETA: 0s - loss: 0.7772 - 

In [ ]:
## With embeddings and numerical features

2/2 [==============================] - 0s 10ms/step - loss: 0.5537 - accuracy: 0.7250 - auc: 0.9781

[INFO] On Training Set:
[0.40510353446006775, 0.8153077960014343, 0.9889481663703918]

[INFO] On Validation Set:
[0.6811900734901428, 0.646766185760498, 0.9671762585639954]

[INFO] On Test Set:
[0.5536547303199768, 0.7250000238418579, 0.9780610799789429]


In [ ]:
dmodel_3_num_emb, dmodel_3_num_emb_path, d3_es, d3_mc = build_deep_model(feature_column_dict,
                                                                         inputs,
                                                                         dmodel_dir,
                                                                         name="dmodel_3_num_emb.h5",
                                                                         ckpt_name="dmodel_3_num_emb_checkpoint.h5")
dmodel_3_num_emb.summary()

In [ ]:
again_training = True
if again_training:
    # create callback for model saving
    d3_m = tf.keras.callbacks.ModelCheckpoint(
        filepath=dmodel_3_num_emb_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        mode='min')
    H3 = dmodel_3_num_emb.fit(train_dl,
                              batch_size=batch_size,
                              epochs=n_epochs,
                              validation_data=val_dl,
                              shuffle=False,
                              validation_batch_size=batch_size,
                              callbacks=[d3_es, d3_mc, d3_m])
else:
    dmodel_3_num_emb = tf.keras.models.load_model(dmodel_3_num_emb_path)

In [ ]:
eval_dmodel_3_num_emb_train = dmodel_3_num_emb.evaluate(train_dl, batch_size=batch_size)
eval_dmodel_3_num_emb_val = dmodel_3_num_emb.evaluate(val_dl, batch_size=batch_size)
eval_dmodel_3_num_emb_test = dmodel_3_num_emb.evaluate(test_dl, batch_size=batch_size)
# Print the results
print("\n[INFO] On Training Set:")
print(eval_dmodel_3_num_emb_train)
print("\n[INFO] On Validation Set:")
print(eval_dmodel_3_num_emb_val)
print("\n[INFO] On Test Set:")
print(eval_dmodel_3_num_emb_test)

In [ ]:
## With normal and hidden numeric features

In [ ]:
# Get the new feature column and input dicts
feature_column_dict_hidden, feature_column_input_dict_hidden = generate_feature_columns(hidden_include=True)
inputs_hidden = {**feature_column_input_dict_hidden["numeric"], **feature_column_input_dict_hidden["embedding"]}
dmodel_4_hid, dmodel_4_hid_path, d4_es, d4_mc = build_deep_model(feature_column_dict_hidden,
                                                                 inputs_hidden,
                                                                 dmodel_dir,
                                                                 name="dmodel_4_hid.h5",
                                                                 ckpt_name="dmodel_4_hid_checkpoint.h5")
dmodel_4_hid.summary()

In [ ]:
again_training = True
if again_training:
    # create callback for model saving
    d4_m = tf.keras.callbacks.ModelCheckpoint(
        filepath=dmodel_4_hid_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        mode='min')
    H4 = dmodel_4_hid.fit(train_dl,
                          batch_size=batch_size,
                          epochs=n_epochs,
                          validation_data=val_dl,
                          shuffle=False,
                          validation_batch_size=batch_size,
                          callbacks=[d4_es, d4_mc, d4_m])
else:
    dmodel_4_hid = tf.keras.models.load_model(dmodel_4_hid_path)

In [ ]:
eval_dmodel_4_hid_train = dmodel_4_hid.evaluate(train_dl, batch_size=batch_size)
eval_dmodel_4_hid_val = dmodel_4_hid.evaluate(val_dl, batch_size=batch_size)
eval_dmodel_4_hid_test = dmodel_4_hid.evaluate(test_dl, batch_size=batch_size)
# Print the results
print("\n[INFO] On Training Set:")
print(eval_dmodel_4_hid_train)
print("\n[INFO] On Validation Set:")
print(eval_dmodel_4_hid_val)
print("\n[INFO] On Test Set:")
print(eval_dmodel_4_hid_test)

In [ ]:
# Wide & deep model

In [ ]:
wdmodel, wdmodel_path, wd_es, wd_mc = build_wide_and_deep_model(feature_column_dict,
                                                                inputs,
                                                                wdmodel_dir=wdmodel_dir)
wdmodel.summary()  # To display the architecture

In [ ]:
again_training = True
if again_training:
    # create callback for model saving
    wd_m = tf.keras.callbacks.ModelCheckpoint(
        filepath=wdmodel_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        mode='min')
    H = wdmodel.fit(train_dl,
                    batch_size=batch_size,
                    epochs=n_epochs,
                    validation_data=val_dl,
                    shuffle=False,
                    validation_batch_size=batch_size,
                    callbacks=[wd_es, wd_mc, wd_m])
else:
    wdmodel = tf.keras.models.load_model(wdmodel_path)

In [ ]:
eval_wdmodel_train = wdmodel.evaluate(train_dl, batch_size=batch_size)
eval_wdmodel_val = wdmodel.evaluate(val_dl, batch_size=batch_size)
eval_wdmodel_test = wdmodel.evaluate(test_dl, batch_size=batch_size)
# Print the results
print("\n[INFO] On Training Set:")
print(eval_wdmodel_train)
print("\n[INFO] On Validation Set:")
print(eval_wdmodel_val)
print("\n[INFO] On Test Set:")
print(eval_wdmodel_test)

In [ ]:
## Bayesian Wide & deep model

In [ ]:
bmodel, bmodel_path, b_es, b_mc = build_bayesian_model(feature_column_dict,
                                                       inputs,
                                                       bayesian_dir)
bmodel.summary()  # To display the architecture

In [ ]:
again_training = True
if again_training:
    # create callback for model saving
    b_m = tf.keras.callbacks.ModelCheckpoint(
        filepath=bmodel_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        mode='min')
    H = bmodel.fit(train_dl,
                   batch_size=batch_size,
                   epochs=n_epochs,
                   validation_data=val_dl,
                   shuffle=False,
                   validation_batch_size=batch_size,
                   callbacks=[b_es, b_mc, b_m])
else:
    bmodel = tf.keras.models.load_model(bmodel_path)

In [ ]:
ts_train, ucb_train = evaluate_bandit(bmodel, train_dl)
ts_val, ucb_val = evaluate_bandit(bmodel, val_dl)
ts_test, ucb_test = evaluate_bandit(bmodel, test_dl)
# Print the results
print("\nUCB\n[INFO] On Training Set:")
print(ucb_train)
print("\n[INFO] On Validation Set:")
print(ucb_val)
print("\n[INFO] On Test Set:")
print(ucb_test)

# Print the results
print("\nThompson Sampling\n[INFO] On Training Set:")
print(ts_train)
print("\n[INFO] On Validation Set:")
print(ts_val)
print("\n[INFO] On Test Set:")
print(ts_test)
